In [2]:
from selenium import webdriver

In [3]:
#imports
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
from collections import deque
from selenium.webdriver.common.keys import Keys
import time


browser = webdriver.Chrome()
browser.get('https://web.telegram.org/#/im')

In [169]:
#global objects

# Home Group
global HOME_GROUP
HOME_GROUP = 'PAID GROUP'

# GroupQueue
global groupQueue
groupQueue = deque([])


#grabs all groupchats
global groupChats
groupChats = browser.find_elements_by_class_name("im_dialog_wrap")

#Selected Top Group
global selectedGroup
selectedGroup = groupChats[0]

#Last Messagege Dictionary
global lastMessageDict
lastMessageDict = {}

#Message Stack
global MESSAGE_STACK
MESSAGE_STACK = []

#Selected group title
global SELECTED_GROUP_TITLE
SELECTED_GROUP_TITLE = ''

In [170]:
#Creates dictionairy of last messages in each group
def initializeDict():
    for chat in groupChats:
        chat.click()
        messages = browser.find_elements_by_class_name("im_message_text")
        textMessages = []
        #change array of WebElements to array of strings. easier to work with, avoid errors
        for webElement in messages:
            textMessages.append(webElement.text)
        #for some reason there were empty strings being grabbed by selenium. removing them here    
        textMessages = list(filter(None, textMessages))
        key_gc = chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        lastMessageDict[key_gc] = textMessages[-1]
# TESTING
initializeDict()
print(lastMessageDict)

{'FGCU CRYPTO CRIPS': 'Futures expires friday', 'Jakub Nowak': 'Jakub Nowak: sry', 'Whale Group': 'Whale Group: test1', 'Telegram': 'OK'}


In [182]:
#groupQueueBuilder
def groupQueueBuilder():
    for groupChat in groupChats:
        oneBadgeArray = groupChat.find_elements_by_css_selector("span.im_dialog_badge.badge")
#         TO DO : make it not take messages from HOME GROUP
        if(oneBadgeArray[0].text!=""):
            #get index and use this index in titlesArray
            oneTitleArray = groupChat.find_elements_by_css_selector("div.im_dialog_peer")
            if(oneTitleArray[0].text not in groupQueue):
                groupQueue.append(oneTitleArray[0].text)
    print(groupQueue)
groupQueueBuilder()

# TESTING
groupQueue.append('Whale Group')

deque([])


In [183]:
#groupFinder - Takes in name of group finds it selects it
def groupFinder(name):
    groupQueue.popleft()
    #print(chats)
    for chat in groupChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == name):
            chat.click()
            global SELECTED_GROUP_TITLE
            SELECTED_GROUP_TITLE = titleArray[0].text
            break;
    
groupFinder(groupQueue[0])
print(groupQueue)

deque([])


In [187]:
#Create push method to click on 'home chat' and paste and send it there.
# checks via dictionary which messages to push
def createMessageStack():
    MESSAGE_STACK.clear()
    messages = browser.find_elements_by_class_name("im_message_text")
    textMessages = []
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    #reversing list to make FIFO
    textMessages.reverse()
    print(lastMessageDict[SELECTED_GROUP_TITLE])
    for message in textMessages:
        
        if(message != lastMessageDict[SELECTED_GROUP_TITLE]):
            MESSAGE_STACK.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            
            break
    print('message Stack:')
    print(MESSAGE_STACK)
        

createMessageStack()


Whale Group: test1
message Stack:
['Jakub Nowak: sry', 'Jakub Nowak: :dancers:♂', 'Jakub Nowak: gay', 'Jakub Nowak: Man I fucking love coding', "Jakub Nowak: When shit works it's just the best fucking feeling", "Jakub Nowak: Do me a favor. While I'm showering watch the 02 video from that video series", "Jakub Nowak: How to get text, I think it's titler", 'Jakub Nowak: Titled', 'Jakub Nowak: He explains promises a bit', 'Jakub Nowak: kk', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]/div[1]/div[2]/div/div/div[2]/div/div[1]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]', '//body/   div[1]/                   div[2]/        div/                        div[2]/                               div[3]/   

In [188]:
# Broadcast Method
def broadCast():
    #select home chat
    print('Will now be sending:')
    print(MESSAGE_STACK)
    for chat in groupChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == HOME_GROUP):
            chat.click()
            break;
    
    #once homechat is selected, send messages one by one.
    textBox = browser.find_elements_by_class_name('composer_rich_textarea')[0]
    for message in MESSAGE_STACK:        
        textBox.send_keys(titleArray[0].text + ": " + message)
        textBox.send_keys(Keys.RETURN)
    lastMessageDict[SELECTED_GROUP_TITLE] = MESSAGE_STACK[-1]
    #clean up after sending messages.
    MESSAGE_STACK.clear()

print(SELECTED_GROUP_TITLE)   
broadCast()

Whale Group
Will now be sending:
['Jakub Nowak: sry', 'Jakub Nowak: :dancers:♂', 'Jakub Nowak: gay', 'Jakub Nowak: Man I fucking love coding', "Jakub Nowak: When shit works it's just the best fucking feeling", "Jakub Nowak: Do me a favor. While I'm showering watch the 02 video from that video series", "Jakub Nowak: How to get text, I think it's titler", 'Jakub Nowak: Titled', 'Jakub Nowak: He explains promises a bit', 'Jakub Nowak: kk', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]/div[1]/div[2]/div/div/div[2]/div/div[1]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[2]', '//*[@id="ng-app"]/body/div[1]/div[2]/div/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/div[2]', '//body/   div[1]/                   div[2]/        div/                        div[2]/                               div[3]/    

In [13]:
#Bot Loop
initializeDict()
while(true):
    if(len(list(groupQueue)) == 0):
        #timeout for x second(s)
        time.sleep(3)
    else:
        print('it is not 0')
        
            #get first in queue and cache / print all messages from it.
            #For our purpose, batch work scheduler would be better. **prioritizes active groupchats**

it is 0
